In [1]:
import os
import shutil
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [2]:
# Diretórios
classificacao_dir = 'image/classificacao'
treinamento_dir = 'image/treinamento'
classes = ["boa", "ruim"]

In [3]:
# Criar diretórios de treinamento se não existirem
for classe in classes:
    os.makedirs(os.path.join(treinamento_dir, classe), exist_ok=True)

In [4]:
# Função para limpar nomes das imagens
def limpar_nome(nome):
    nome = unicodedata.normalize('NFKD', nome).encode('ASCII', 'ignore').decode('ASCII')
    nome = nome.replace("Copia de ", "").replace("Cópia de ", "").replace(" ", "")
    return nome

In [5]:
# Renomear imagens em todas as subpastas
def renomear_imagens_em_todas(diretorio_principal):
    for classe in classes:
        pasta = os.path.join(diretorio_principal, classe)
        if not os.path.isdir(pasta):
            continue
        for nome_antigo in os.listdir(pasta):
            caminho_antigo = os.path.join(pasta, nome_antigo)
            if os.path.isfile(caminho_antigo) and nome_antigo.lower().endswith(('.jpg', '.jpeg', '.png')):
                nome_limpo = limpar_nome(nome_antigo)
                caminho_novo = os.path.join(pasta, nome_limpo)
                if caminho_antigo != caminho_novo:
                    os.rename(caminho_antigo, caminho_novo)
                    print(f"Renomeado: {nome_antigo} → {nome_limpo}")

# Executar renomeio
renomear_imagens_em_todas(classificacao_dir)

In [6]:
# Função para emparelhar imagens consecutivas (ex: SNAP0000.jpg + SNAP0001.jpg)
def emparelhar_imagens(origem_dir):
    imagens = sorted([f for f in os.listdir(origem_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    pares = []
    for i in range(0, len(imagens) - 1, 2):  # Pega dois de cada vez
        img1 = os.path.join(origem_dir, imagens[i])
        img2 = os.path.join(origem_dir, imagens[i + 1])
        pares.append([img1, img2])
    return pares

In [7]:
for classe in classes:
    origem = os.path.join(classificacao_dir, classe)
    destino = os.path.join(treinamento_dir, classe)
    os.makedirs(destino, exist_ok=True)

    pares = emparelhar_imagens(origem)

    for par in pares:
        for img_path in par:
            nome_arquivo = os.path.basename(img_path)
            nome_limpo = limpar_nome(nome_arquivo)
            novo_caminho = os.path.join(destino, nome_limpo)

            # Tenta carregar a imagem antes de copiar
            img = cv2.imread(img_path)
            if img is not None:
                shutil.copy(img_path, novo_caminho)
            else:
                print(f"Imagem ignorada por erro de leitura: {img_path}")

In [8]:
def processar_par_de_imagens(par):
    img1 = cv2.imread(par[0])
    img2 = cv2.imread(par[1])
    if img1 is None:
        raise ValueError(f"Erro ao carregar a imagem: {par[0]}")
    if img2 is None:
        raise ValueError(f"Erro ao carregar a imagem: {par[1]}")
    img1 = cv2.resize(img1, (128, 128))
    img2 = cv2.resize(img2, (128, 128))
    img_concatenada = np.concatenate((img1, img2), axis=1)  # Junta lado a lado
    return img_concatenada

In [9]:
# Criar gerador customizado para carregar pares de imagens
class ParImageDataGenerator:
    def __init__(self, directory, batch_size=32, target_size=(128, 256), class_mode='sparse'):
        self.directory = directory
        self.batch_size = batch_size
        self.target_size = target_size
        self.class_mode = class_mode
        self.classes = classes
        self.data = self._load_data()

    def _load_data(self):
        data = []
        for classe in self.classes:
            classe_dir = os.path.join(self.directory, classe)
            pares = emparelhar_imagens(classe_dir)
            for par in pares:
                data.append((par, self.classes.index(classe)))
        return data

    def __iter__(self):
        return self

    def __next__(self):
        if not self.data:
            raise StopIteration
        batch = self.data[:self.batch_size]
        self.data = self.data[self.batch_size:]
        inputs = [processar_par_de_imagens(par) for par, _ in batch]
        labels = [label for _, label in batch]
        return np.array(inputs) / 255.0, np.array(labels)

In [10]:
# Criar geradores de treinamento e validação
train_generator = ParImageDataGenerator(treinamento_dir)
validation_generator = ParImageDataGenerator(treinamento_dir)

In [11]:
# Caminho do modelo
modelo_path = 'model/modelo_solda.h5'

In [12]:
# Verifica se o modelo já existe
if os.path.exists(modelo_path):
    model = load_model(modelo_path)
    print("Modelo carregado, continuando o treinamento...")
else:
    print("Treinando um novo modelo...")
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 256, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Treinando um novo modelo...


c:\Users\finan\OneDrive - Nimofast Brasil SA\Área de Trabalho\cod\python\AI\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# Verificar se há dados antes de treinar
try:
    x_train, y_train = next(iter(train_generator))
    x_val, y_val = next(iter(validation_generator))

    model.fit(
        x=x_train, y=y_train,
        epochs=10,
        validation_data=(x_val, y_val)
    )

    model.save(modelo_path)
    print(f"Modelo salvo em: {modelo_path}")

except StopIteration:
    print("Erro: Nenhum dado disponível para treino. Verifique as imagens no diretório.")

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.9062 - loss: 0.6510 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 536ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 676ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 702ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 719ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 686ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 609ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 618ms/step - accuracy: 1.0000 

Modelo salvo em: model/modelo_solda.h5
